<a id="top"></a>
# ULLYSES Data Walkthrough Notebook

***

## Learning Goals

By the end of this tutorial, you will:

- Know the different types of high level science products (HLSPs) offered by the ULLYSES team.
- Be able to open each file and view its contents and metadata.
- Understand how to use each type of HSLP.

## Table of Contents
**0. [Introduction](#introduction)**

**1. [HLSP Descriptions and Filename Convention](#hlspdescrip)**

**2. [HLSP File Exploration](#hlspformats)**

\- 2.1 [Single-epoch Spectra](#single)

\- 2.2 [Time-series Spectra](#tss)

\- 2.3 [WFC3 Drizzled Images](#drizzle)

**3. [Examples](#examples)**

\- 3.1 [Example 1: Exploring HLSPs](#ex1)

\- 3.2 [Example 2: Getting HLSP Metadata](#ex2)

\- 3.3 [Example 3: ](#ex3)

<a id="introduction"></a>
## Introduction

The Hubble Space Telescope’s (HST) Ultraviolet Legacy Library of Young Stars as Essential Standards ([ULLYSES](https://ullyses.stsci.edu/index.html) program has devoted approximately 1,000 HST orbits to the production of an ultraviolet spectroscopic library of young high- and low-mass stars in the local universe. This Director’s Discretionary program was designed to take advantage of HST’s unique UV capabilities, as both high- and low-mass stars feature different complex UV emission processes that strongly impact their surroundings, but are difficult to model. The UV emission from star formation is central to a wide range of vital astrophysical issues, ranging from cosmic reionization to the formation of planets.

The ULLYSES program uniformly sampled the fundamental astrophysical parameter space for each mass regime — including spectral type, luminosity class, and metallicity for massive OB stars (in the Magellanic Clouds and two other lower-metallicity nearby galaxies) and the mass, age, and disk accretion rate for low-mass T Tauri stars (in eight young Galactic associations). The data were gathered over a three-year period, from Cycle 27 through Cycle 29 (2020-2022). The design and targets of these observations were determined in partnership with the astronomical community, allowing researchers from around the world to help develop the final program and to plan coordinated observations with other space- and ground-based telescopes.

The ULLYSES team produces several types of High Level Science Products (HLSPs), which we will walk through in this notebook. Products are made using both archival data and new HST observations obtained through the ULLYSES program. Data products are available from the [ULLYSES website](https://ullyses.stsci.edu/ullyses-download.html) (HLSPs and contributing data), the [MAST Data Discovery Portal](https://mast.stsci.edu/) (HLSPs and contributing data), or directly as a High-Level Science Product collection using the [DOI](https://archive.stsci.edu/hlsp/ullyses) (HLSPs only).

### Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data
- *Path* to create product and data directories
- *shutil* to perform directory and file operations
- *os* to interact with the operating system

In [2]:
%matplotlib inline
import numpy as np
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import os
from pathlib import Path
import shutil

***

<a id="hlspdescrip"></a>
## HLSP Descriptions and Filename Convention

The file names for ULLYSES science data products follow a naming scheme which encodes the target designation and the instruments and observing configuration(s) that contribute to the product. However, not all products will appear in the early releases. File names have the form:

`hlsp_ullyses_<telescope>_<instrument>_<target>_<opt_elem>_<version>_<product-type>`

where

`<target>` is the target name
`<version>` is the data release identifier (dr1, dr2, etc.)

The `<telescope>`, `<instrument>`, `<opt_elem>`, and `<product-type>` templates take names from the following table:

| Description                                                                                                  | Telescope | Instrument    | Opt-Elem                                        | Product-Type      | Level |
|--------------------------------------------------------------------------------------------------------------|-----------|---------------|-------------------------------------------------|-------------------|-------|
| Custom calibrated STIS 1D spectra                                                                            | hst       | stis          | g230l, g430l, or g750l                             | spec.fits         | 0     |
| STIS custom calibration parameter files                                                                      | hst       | stis          | g230l, g430l, or g750l                             | spec.yaml         | 0     |
| STIS echelle single grating, where the orders have been extracted and merged.                                | hst       | stis          | e140l, e230h, e140m, or e230m                      | mspec.fits        | 1     |
| Combined spectra, with common instrument and  grating, and in some cases with different cenwave settings.    | hst       | cos           | g130m, g160m, g185m, or g230l                      | cpsec.fits        | 2     |
|                                                                                                              |           | stis          | e140l, e230h, e140m, e230m, g230l, g430l, or g750l |                   |       |
| Combined spectra, with common instrument, different gratings and cenwave settings, and grouped by resolution | fuse      | fuv           | lwrs or mdrs                                    | cspec.fits        | 3     |
|                                                                                                              | hst       | cos           | g130m-g160m-g185m                               |                   |       |
|                                                                                                              |           | stis          | e140h-e230h, g140m-e230m or g230l-g430l-g750l |                   |       |
| All instruments and settings abutted together*                                                               | hst       | cos-stis      | uv or uv-opt                                    | preview-spec.fits | 4     |
|                                                                                                              | hst-fuse  | fuse-cos-stis | uv-opt                                          |                   |       |
| Exposure-level time-series spectra                                                                           | hst       | cos           | g130m or g160m                                    | tss.fits          | 5     |
|                                                                                                              | lcogt     | 04m           | v-ip                                            |                   |       |
| Subexposure-level time-series spectra                                                                        | hst       | cos           | g130m or g160m                                    | split-tss.fits    | 5     |
| WFC3 drizzled images                                                                                         | hst       | wfc3          | f225w, f275w, f336w, f475w or f814w               | drc.fits          | 6     |

***

<a id="hlspformats"></a>
## HLSP File Formats

Most High Level Science Products are in FITS format. The organization of each FITS file depends on the HLSP type. There are three broad categories of HLSPs: single-epoch spectra, time-series spectra, and drizzled images.

<a id="single"></a>
### Single Epoch Spectra

#### FITS File Structure

Spectral data and information is stored in two BINTABLE extensions:
- a Science extension for the combined product, and
- a matching Provenance extension to record attributes of each spectrum that contributed to the combined product

<a id="tss"></a>
### Timeseries Spectra

#### FITS File Structure

Like the single epoch spectra, the time series spectral (TSS) data and information is stored in two BINTABLE extensions:
- a Science extension for the combined product, and
- a matching Provenance extension to record attributes of each spectrum that contributed to the combined product

Next, let's download a TTS file and explore.

In [7]:
# Use curl to download a TSS data product for the target Hn5 (GX Cha)
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fhn5%2Fdr6%2Fhlsp_ullyses_lcogt_04m_hn5_v-iprime_dr6_tss.fits" --output "hlsp_ullyses_lcogt_04m_hn5_v-iprime_dr6_tss.fits" --fail --create-dirs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 54720  100 54720    0     0   269k      0 --:--:-- --:--:-- --:--:--  276k


In [10]:
# Create a variable to point to the file we just downloaded
tss_file = Path('./hlsp_ullyses_lcogt_04m_hn5_v-iprime_dr6_tss.fits')
tss_data = fits.open(tss_file)
print(tss_data.info())

Filename: hlsp_ullyses_lcogt_04m_hn5_v-iprime_dr6_tss.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      36   ()      
  1  SCIENCE       1 BinTableHDU     36   1R x 5C   [100D, 100D, 2E, 200E, 200E]   
  2  PROVENANCE    1 BinTableHDU     37   100R x 12C   [40A, 32A, 32A, 32A, 32A, 32A, 32A, 32A, D, D, D, D]   
None


\
From the filename, we can see:
- the telescope used is LCOGT (add link here)
- the instrument used is the 0.4 meter
- the optical element is v-iprime
- this file was created for data release 6

Now we'll look at the science and provenance extensions.

#### Time-series Spectra Science Table
The TSS products have slightly different table columns compared to single-epoch spectra. The FLUX and ERROR arrays are 2D arrays with wavelength increasing X, and time increasing along Y. The wavelength values for each column of the 2D data are stored in the WAVELENGTH array, while the MJDSTART and MJDEND columns store the start and end times for each row of the FLUX and ERROR arrays.

In [25]:
# Print the columns in the science extension
print(tss_data[1].columns)

# Print the shape of the FLUX array
print('Number of time bins: {}'.format(tss_data[1].data['FLUX'][0].shape[0]))
print('Number of spectral bins: {}'.format(tss_data[1].data['FLUX'][0].shape[1]))

ColDefs(
    name = 'MJDSTART'; format = '100D'; unit = 'Day'
    name = 'MJDEND'; format = '100D'; unit = 'Day'
    name = 'WAVELENGTH'; format = '2E'; unit = 'Angstrom'
    name = 'FLUX'; format = '200E'; unit = 'erg /s /cm**2 /Angstrom'; dim = '(2, 100)'
    name = 'ERROR'; format = '200E'; unit = 'erg /s /cm**2 /Angstrom'; dim = '(2, 100)'
)
Number of time bins: 100
Number of spectral bins: 2


In [ ]:
# Plot the flux vs wavelength over time



#### Time-series Spectra Provenance Table
Select metadata for each spectrum that contributes to the time-series product will populate a row in the provenance table. The fields in the table are metadata harvested from the headers of the contributing spectra. Some columns are only present for HST data or LCOGT data.

In [28]:
# Print the columns in the provenance extension
print(tss_data[2].columns)

ColDefs(
    name = 'FILENAME'; format = '40A'
    name = 'PROPOSID'; format = '32A'
    name = 'TELESCOPE'; format = '32A'
    name = 'INSTRUMENT'; format = '32A'
    name = 'DETECTOR'; format = '32A'
    name = 'FILTER'; format = '32A'
    name = 'APERTURE'; format = '32A'
    name = 'CAL_VER'; format = '32A'
    name = 'MJD_BEG'; format = 'D'; unit = 'd'
    name = 'MJD_MID'; format = 'D'; unit = 'd'
    name = 'MJD_END'; format = 'D'; unit = 'd'
    name = 'XPOSURE'; format = 'D'; unit = 's'
)


In [27]:
# Print a list of the filenames used to create this data product
print(tss_data[2].data['FILENAME'])

['coj0m405-kb88-20210307-0098-e91.fits.fz'
 'lsc0m409-kb29-20210310-0115-e91.fits.fz'
 'coj0m403-kb24-20210312-0079-e91.fits.fz'
 'lsc0m409-kb29-20210314-0117-e91.fits.fz'
 'coj0m403-kb24-20210315-0118-e91.fits.fz'
 'coj0m403-kb24-20210316-0099-e91.fits.fz'
 'coj0m403-kb24-20210521-0080-e91.fits.fz'
 'coj0m403-kb24-20210521-0081-e91.fits.fz'
 'coj0m403-kb24-20210522-0067-e91.fits.fz'
 'coj0m403-kb24-20210522-0068-e91.fits.fz'
 'lsc0m409-kb29-20210524-0076-e91.fits.fz'
 'lsc0m409-kb29-20210525-0063-e91.fits.fz'
 'lsc0m409-kb29-20210526-0061-e91.fits.fz'
 'coj0m403-kb24-20210528-0123-e91.fits.fz'
 'lsc0m409-kb29-20210528-0068-e91.fits.fz'
 'lsc0m409-kb29-20210529-0126-e91.fits.fz'
 'coj0m405-kb88-20210530-0073-e91.fits.fz'
 'coj0m405-kb88-20210530-0074-e91.fits.fz'
 'lsc0m409-kb29-20210530-0066-e91.fits.fz'
 'lsc0m409-kb29-20210530-0118-e91.fits.fz'
 'lsc0m409-kb29-20210530-0130-e91.fits.fz'
 'lsc0m409-kb29-20210530-0136-e91.fits.fz'
 'lsc0m409-kb29-20210530-0142-e91.fits.fz'
 'lsc0m409-

<a id="drizzle"></a>
### WFC3 Drizzled Images

#### FITS File Structure

Drizzled and weight images are stored in two IMAGEHDU extensions, and PROVENANCE information stored in a BINTABLE extension:
- a Science extension for the drizzled science image
- a Weight extension for the weight map image
- a matching Provenance extension to record attributes of each image that contributed to the combined product

***

<a id="examples"></a>
## Examples

<a id="ex1"></a>
### Example 1

<a id="ex2"></a>
### Example 2

<a id="ex3"></a>
### Example 3

***

## Additional Resources

This section is optional. Try to weave resource links into the main content of your tutorial so that they are falling in line with the context of your writing. For resources that do not fit cleanly into your narrative, you may include an additional resources section at the end of your tutorial. Usually a list of links using Markdown bullet list plus link format is appropriate:

- [MAST API](https://mast.stsci.edu/api/v0/index.html)
- [Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)
- [Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf)
- [Exo.MAST website](https://exo.mast.stsci.edu/)

## About this Notebook
Let the world know who the author of this great tutorial is! If possible and appropriate, include a contact email address for users who might need support (for example, `archive@stsci.edu`). You can also optionally include keywords, your funding source, or a last update date in this section.

**Author:** Jessie Blogs, Archive Scientist.  
**Updated On:** YYYY-MM-DD

## Citations
Provide your reader with guidelines on how to cite open source software and other resources in their own published work.

```
If you use `astropy` or `lightkurve` for published research, please cite the
authors. Follow these links for more information about citing `astropy` and
`lightkurve`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `lightkurve`](http://docs.lightkurve.org/about/citing.html)

```

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 